### 0. Spark Setup

In [ ]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-01.csv.gz
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6ff17bf4-c38d-4f44-ac31-0c79953cb01d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

23/03/17 21:32:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. Reading from Kafka Stream

through `readStream`

#### 1.1 Raw Kafka Stream

In [3]:
# default for startingOffsets is "latest"
df_kafka_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "rides_csv") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

In [4]:
df_kafka_raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



#### 1.2 Encoded Kafka Stream

In [5]:
df_kafka_encoded = df_kafka_raw.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [6]:
df_kafka_encoded.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



#### 1.3 Structure Streaming DataFrame

In [7]:
def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column
    col = F.split(df['value'], ', ')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

In [8]:
ride_schema = T.StructType(
    [T.StructField('pickup_datetime', T.TimestampType()),
     T.StructField('dropoff_datetime', T.TimestampType()),
     T.StructField("pickup_loc_id", T.IntegerType()),
     T.StructField("dropoff_loc_id", T.IntegerType())
     ])

In [9]:
df_rides = parse_ride_from_kafka_message(df_raw=df_kafka_raw, schema=ride_schema)

In [10]:
df_rides.printSchema()

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- pickup_loc_id: integer (nullable = true)
 |-- dropoff_loc_id: integer (nullable = true)



### 2 Sink Operation & Streaming Query

through `writeStream`

---
**Output Sinks**
- File Sink: stores the output to the directory
- Kafka Sink: stores the output to one or more topics in Kafka
- Foreach Sink:
- (for debugging) Console Sink, Memory Sink

Further details can be found in [Output Sinks](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-sinks)

---
There are three types of **Output Modes**:
- Complete: The whole Result Table will be outputted to the sink after every trigger. This is supported for aggregation queries.
- Append (default): Only new rows are added to the Result Table
- Update: Only updated rows are outputted

[Output Modes](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-modes) differs based on the set of transformations applied to the streaming data. 

--- 
**Triggers**

The [trigger settings](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#triggers) of a streaming query define the timing of streaming data processing. Spark streaming support micro-batch streamings schema and you can select following options based on requirements.

- default-micro-batch-mode
- fixed-interval-micro-batch-mode
- one-time-micro-batch-mode
- available-now-micro-batch-mode


#### Console and Memory Sink

In [11]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [12]:
write_query = sink_console(df_rides, output_mode='append')

23/03/17 21:33:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-be98cbad-d182-484f-9c12-fa9c91419679. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/17 21:33:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/17 21:33:34 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-47615cb7-0a0a-45c3-80e4-701ef6175562-10950832-driver-0-1, groupId=spark-kafka-source-47615cb7-0a0a-45c3-80e4-701ef6175562-10950832-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/03/17 21:33:34 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-47615cb7-0a0a-45c3-80e4-701ef6175562-10950832-driver-0-1

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-------------------+-------------+--------------+
|pickup_datetime    |dropoff_datetime   |pickup_loc_id|dropoff_loc_id|
+-------------------+-------------------+-------------+--------------+
|2018-12-21 15:17:29|2018-12-21 15:18:57|264          |264           |
|2019-01-01 00:10:16|2019-01-01 00:16:32|97           |49            |
|2019-01-01 00:27:11|2019-01-01 00:31:38|49           |189           |
|2019-01-01 00:46:20|2019-01-01 01:04:54|189          |17            |
|2019-01-01 00:19:06|2019-01-01 00:39:43|82           |258           |
+-------------------+-------------------+-------------+--------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+-------------------+-------------+--------------+
|pickup_datetime    |dropoff_datetime   |pickup_loc_id|dropoff_loc_id|
+-------------------+-------------------+-------------+--------------+
|2019-01-01 00:30:00|2019-01-01 02:51:55|null         |null          |
|2019-01-01 00:45:00|2019-01-01 00:54:49|null         |null          |
|2019-01-01 00:15:00|2019-01-01 00:54:52|null         |null          |
|2019-01-01 00:19:00|2019-01-01 00:39:00|null         |null          |
|2019-01-01 00:27:00|2019-01-01 00:37:00|null         |null          |
+-------------------+-------------------+-------------+--------------+



In [19]:
def sink_memory(df, query_name, query_template):
    write_query = df \
        .writeStream \
        .queryName(query_name) \
        .format('memory') \
        .start()
    query_str = query_template.format(table_name=query_name)
    query_results = spark.sql(query_str)
    return write_query, query_results

In [20]:
query_name = 'pickup_loc_counts'
query_template = 'select pickup_loc_id, count(pickup_datetime) from {table_name} group by pickup_loc_id'
write_query, df_pickup_loc_counts = sink_memory(df=df_rides, query_name=query_name, query_template=query_template)

23/03/17 21:37:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3b9301b3-adeb-40e9-8667-be9d5efed5ff. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/17 21:37:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/17 21:37:20 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-74cef17f-69f2-41ac-82c3-a8c5f2b83dc3--480960087-executor-6, groupId=spark-kafka-source-74cef17f-69f2-41ac-82c3-a8c5f2b83dc3--480960087-executor] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/03/17 21:37:20 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-74cef17f-69f2-41ac-82c3-a8c5f2b83dc3--480960087-exec

In [21]:
print(type(write_query)) # pyspark.sql.streaming.StreamingQuery
write_query.status

<class 'pyspark.sql.streaming.StreamingQuery'>


{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [22]:
df_pickup_loc_counts.show()

+-------------+----------------------+
|pickup_loc_id|count(pickup_datetime)|
+-------------+----------------------+
|           49|                     2|
|           97|                     2|
|          264|                     2|
|         null|                     5|
|          189|                     2|
|           82|                     2|
+-------------+----------------------+



In [18]:
write_query.stop()

#### Kafka Sink

To write stream results to `kafka-topic`, the stream dataframe has at least a column with name `value`.

Therefore before starting `writeStream` in kafka format, dataframe needs to be updated accordingly.

More information regarding kafka sink expected data structure [here](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html#writing-data-to-kafka)


In [ ]:
def prepare_dataframe_to_kafka_sink(df, value_columns, key_column=None):
    columns = df.columns
    df = df.withColumn("value", F.concat_ws(', ',*value_columns))    
    if key_column:
        df = df.withColumnRenamed(key_column,"key")
        df = df.withColumn("key",df.key.cast('string'))
    return df.select(['key', 'value'])
    
def sink_kafka(df, topic, output_mode='append'):
    write_query = df.writeStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
        .outputMode(output_mode) \
        .option("topic", topic) \
        .option("checkpointLocation", "checkpoint") \
        .start()
    return write_query